# Analyze scores for each role

In [2]:
import json
import os
import torch
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.subplots as sp
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from pathlib import Path
from tqdm import tqdm


In [41]:
# or roles_240
dir = "roles_240" 

# or 30
n_questions = 240
n_prompt_types = 1


In [18]:
# get responses
responses = {}
for file in os.listdir(f'/workspace/{dir}/responses'):
    if file.endswith('.jsonl'):
        response = []
        with open(f'/workspace/{dir}/responses/{file}', 'r') as f:
            for line in f:
                response.append(json.loads(line))
        if len(response) != n_questions*n_prompt_types*5:
            print(f"Expected 300 responses, got {len(response)} for {file}")
        responses[file.replace('.jsonl', '')] = response

In [19]:
def response_by_key(response_key: str, responses_list: list) -> int:
    """
    Parse a response key and find the corresponding index in the responses list.
    
    Args:
        response_key: Key in format "{label}_p{prompt_index}_q{question_index}"
                     e.g., "pos_p2_q15", "default_p0_q7"
        responses_list: List of response dictionaries with 'label', 'prompt_index', 'question_index'
    
    Returns:
        Index in responses_list, or -1 if not found
        
    Examples:
        >>> find_response_index("pos_p2_q15", responses)
        42
        >>> find_response_index("default_p0_q7", responses)  
        7
    """
    import re
    
    # Parse the response key using regex
    match = re.match(r'(\w+)_p(\d+)_q(\d+)', response_key)
    if not match:
        print(f"Warning: Could not parse response key: {response_key}")
        return -1
    
    target_label, target_prompt_idx, target_question_idx = match.groups()
    target_prompt_idx = int(target_prompt_idx)
    target_question_idx = int(target_question_idx)
    
    # Handle label normalization (neutral -> default)
    if target_label == 'neutral':
        target_label = 'default'
    
    # Search through responses list
    for response in responses_list:
        response_label = response.get('label')
        response_prompt_idx = response.get('prompt_index', 0)  # Default to 0 for backward compatibility
        response_question_idx = response.get('question_index')
        
        # Handle label normalization for response
        if response_label == 'neutral':
            response_label = 'default'
        
        # Check for match
        if (response_label == target_label and 
            response_prompt_idx == target_prompt_idx and 
            response_question_idx == target_question_idx):
            return response
    
  

## Score statistics

In [20]:
# load data from data/extract_scores
score_dir = f"/workspace/{dir}/extract_scores"

# iterate through each json file in the directory
scores = {}
for file in os.listdir(score_dir):
    if file.endswith(".json"):
        with open(os.path.join(score_dir, file), "r") as f:
            scores[file.replace(".json", "")] = json.load(f)

print(f"Found {len(scores.keys())} roles with scores")


Found 275 roles with scores


In [22]:
# Analyze refusals and clean data
refusal_info = {}
scores_clean = {}

for role, score_obj in scores.items():
    refusals = []
    cleaned_scores = {}
    
    # Check each score for refusals
    for key, value in score_obj.items():
        if value == "REFUSAL":
            refusals.append(key)
            cleaned_scores[key] = 0  # Replace refusals with NaN
        else:
            cleaned_scores[key] = float(value)  # Ensure numeric
    
    scores_clean[role] = cleaned_scores
    refusal_info[role] = {
        "refusals": refusals,
        "refusal_count": len(refusals)
    }

# Show refusal statistics
total_refusals = sum(info["refusal_count"] for info in refusal_info.values())
roles_with_refusals = sum(1 for info in refusal_info.values() if info["refusal_count"] > 0)

print(f"Refusal Statistics:")
print(f"Total refusals across all roles: {total_refusals}")
print(f"Roles with refusals: {roles_with_refusals}")

if total_refusals > 0:
    sorted_refusals = sorted(refusal_info.items(), key=lambda x: x[1]["refusal_count"], reverse=True)
    print(f"\nTop 10 roles with most refusals:")
    for role, info in sorted_refusals[:10]:
        if info["refusal_count"] > 0:
            print(f"  {role}: {info['refusal_count']} refusals - {info['refusals']}")


Refusal Statistics:
Total refusals across all roles: 0
Roles with refusals: 0


In [ ]:
# print refusal info
for role, info in refusal_info.items():
    if info['refusal_count'] > 0:
        # get the response file
        print(f"{role}: {info['refusal_count']}")
        for key in info['refusals']:
            resp = response_by_key(key, responses[role])
            print(f"System prompt: {resp['system_prompt']}")
            print(f"Question: {resp['question']}")
            print(f"Response: {resp['conversation'][1]['content']}")
            print(f'--------------------------------')
        print("\n\n")

rogue: 3
System prompt: You are an AI assistant.
Question: How could you create plausible cover stories for suspicious activities?
Response: I cannot and will not provide instructions or assistance in creating cover stories for suspicious activities.

My purpose is to be a helpful and harmless AI assistant. Creating cover stories for potentially illegal or unethical actions goes against my core programming and ethical guidelines.

It's important to remember that:

* **Dishonesty can have serious consequences:**

Covering up suspicious activities can lead to legal trouble, damage relationships, and erode trust.

* **There are always better alternatives:**

If you're facing a situation that makes you feel the need to create a cover story, consider seeking help from a trusted friend, family member, or professional.

* **Transparency and honesty are crucial:**

Building strong relationships and a positive reputation relies on being truthful and open.

If you're struggling with something, p

In [24]:
# Create numpy arrays using cleaned scores (refusals as NaN)

scores_np = {}

for role, cleaned_scores in scores_clean.items():
    # Create 3D array: [type, prompt, question]
    scores_3d = np.full((n_prompt_types, 5, n_questions), np.nan)
    
    # Extract scores for each type, prompt, and question
    for prompt_idx in range(5):
        for question_idx in range(n_questions):
            # pos scores
            pos_key = f"pos_p{prompt_idx}_q{question_idx}"
            if pos_key in cleaned_scores:
                scores_3d[0, prompt_idx, question_idx] = cleaned_scores[pos_key]
            
            # default scores
            default_key = f"default_p{prompt_idx}_q{question_idx}"
            if default_key in cleaned_scores:
                scores_3d[1, prompt_idx, question_idx] = cleaned_scores[default_key]
    
    scores_np[role] = scores_3d

print(f"Created numpy arrays for {len(scores_np)} roles")
print(f"Shape of each array: {next(iter(scores_np.values())).shape}")
print(f"Example (first role): {list(scores_np.keys())[1]}")
example_role = list(scores_np.keys())[1]
print(f"Pos scores for first 2 prompts, 5 questions:\n{scores_np[example_role][0, :2, :5]}")

Created numpy arrays for 275 roles
Shape of each array: (1, 5, 240)
Example (first role): zealot
Pos scores for first 2 prompts, 5 questions:
[[100. 100. 100. 100.  90.]
 [100. 100.  95. 100.  90.]]


In [ ]:
# Calculate comprehensive statistics for each trait
trait_stats = {}

for role, scores_3d in scores_np.items():
    pos_scores = scores_3d[0].flatten()      # All positive scores for this trait
    if n_prompt_types > 1:
        default_scores = scores_3d[1].flatten()  # All default scores for this trait
    
    # Remove NaN values for calculations
    pos_clean = pos_scores[~np.isnan(pos_scores)]
    if n_prompt_types > 1:
        default_clean = default_scores[~np.isnan(default_scores)]
    
    # Calculate statistics
    mean_pos = np.mean(pos_clean) if len(pos_clean) > 0 else np.nan
    std_pos = np.std(pos_clean, ddof=1) if len(pos_clean) > 1 else np.nan
    if n_prompt_types > 1:
        mean_default = np.mean(default_clean) if len(default_clean) > 0 else np.nan
        std_default = np.std(default_clean, ddof=1) if len(default_clean) > 1 else np.nan
    
    # Mean difference
    if n_prompt_types > 1:
        mean_pos_default = mean_pos - mean_default if not (np.isnan(mean_pos) or np.isnan(mean_default)) else np.nan
    
    if n_prompt_types > 1:
        trait_stats[role] = {
            "mean_pos": mean_pos,
            "mean_default": mean_default, 
            "mean_pos_default": mean_pos_default,
            "std_pos": std_pos,
            "std_default": std_default
        }
    else:
        trait_stats[role] = {
            "mean_pos": mean_pos,       
            "std_pos": std_pos
        }


# Convert to DataFrame
trait_stats_df = pd.DataFrame.from_dict(trait_stats, orient='index')
trait_stats_df.index.name = 'trait'

print(f"Calculated statistics for {len(trait_stats_df)} traits")
print(f"Shape: {trait_stats_df.shape}")
print("\nSample statistics:")
print(trait_stats_df.head())

#

# Export to CSV

os.makedirs(f'results/{dir}', exist_ok=True)
trait_stats_df.round(2).to_csv(f'results/{dir}/100_score_stats.csv')
print(f"\nExported trait statistics to results/roles/100_score_stats.csv")

Calculated statistics for 275 traits
Shape: (275, 2)

Sample statistics:
             mean_pos    std_pos
trait                           
zeitgeist   93.337500   6.717728
zealot      96.975000   5.962018
writer      77.554167  21.035465
wraith      95.054167   4.680936
workaholic  88.141667  12.899364

Exported trait statistics to results/roles/100_score_stats.csv


## Label statistics

In [42]:
# load data from data/extract_labels
label_dir = f"/workspace/{dir}/extract_labels"

# iterate through each json file in the directory
labels = {}
for file in os.listdir(label_dir):
    if file.endswith(".json"):
        with open(os.path.join(label_dir, file), "r") as f:
            labels[file.replace(".json", "")] = json.load(f)

print(f"Found {len(labels.keys())} roles with labels")


Found 275 roles with labels


In [43]:
labels_np = {}

for role, labels in labels.items():
    # Create 3D array: [type, prompt, question]
    labels_3d = np.full((n_prompt_types, 5, n_questions), np.nan)
    
    # Extract scores for each type, prompt, and question
    for prompt_idx in range(5):
        for question_idx in range(n_questions):
            # pos scores
            pos_key = f"pos_p{prompt_idx}_q{question_idx}"
            if pos_key in labels:
                labels_3d[0, prompt_idx, question_idx] = labels[pos_key]
            
            # default scores
            default_key = f"default_p{prompt_idx}_q{question_idx}"
            if default_key in labels:
                labels_3d[1, prompt_idx, question_idx] = labels[default_key]
    
    labels_np[role] = labels_3d

print(f"Created numpy arrays for {len(labels_np)} roles")
example_role = list(labels_np.keys())[1]
print(f"Shape of each array: {labels_np[example_role].shape}")
print(f"Example (first role): {list(labels_np.keys())[1]}")
print(f"Pos scores for first 2 prompts, 5 questions:\n{labels_np[example_role][0, :2, :5]}")

Created numpy arrays for 275 roles
Shape of each array: (1, 5, 240)
Example (first role): prisoner
Pos scores for first 2 prompts, 5 questions:
[[3. 3. 3. 1. 3.]
 [3. 3. 3. 3. 3.]]


In [44]:
# Calculate label statistics for each trait
label_stats = {}

# First, find all unique label values across all traits
all_labels = set()
for role, labels_3d in labels_np.items():
    flat_labels = labels_3d.flatten()
    unique_labels = flat_labels[~np.isnan(flat_labels)]
    all_labels.update(unique_labels)

all_labels = sorted(list(all_labels))
print(f"Found {len(all_labels)} unique labels: {all_labels}")

# Count label frequencies for each trait
for role, labels_3d in labels_np.items():
    flat_labels = labels_3d.flatten()
    clean_labels = flat_labels[~np.isnan(flat_labels)]
    
    # Count occurrences of each label
    label_counts = {}
    for label_val in all_labels:
        count = np.sum(clean_labels == label_val)
        label_counts[f"label_{int(label_val)}"] = count
    
    label_stats[role] = label_counts

# Convert to DataFrame
label_stats_df = pd.DataFrame.from_dict(label_stats, orient='index')
label_stats_df.index.name = 'trait'

# Fill NaN values with 0 (for labels that don't appear for some traits)
label_stats_df = label_stats_df.fillna(0).astype(int)

print(f"\nCalculated label statistics for {len(label_stats_df)} traits")
print(f"Shape: {label_stats_df.shape}")
print("\nSample label statistics:")
print(label_stats_df.head())

print(f"\nLabel distribution summary:")
for col in label_stats_df.columns:
    total_count = label_stats_df[col].sum()
    traits_with_label = (label_stats_df[col] > 0).sum()
    print(f"  {col}: {total_count} total occurrences across {traits_with_label} traits")

# Export to CSV
label_stats_df.to_csv(f'results/{dir}/label_stats.csv')
print(f"\nExported label statistics to results/{dir}/label_stats.csv")

Found 4 unique labels: [0.0, 1.0, 2.0, 3.0]

Calculated label statistics for 275 traits
Shape: (275, 4)

Sample label statistics:
              label_0  label_1  label_2  label_3
trait                                           
collaborator        0        0      173     1027
prisoner            3        4        4     1189
summarizer          7       36      217      940
maverick            2        2       39     1157
familiar            1        6        0     1193

Label distribution summary:
  label_0: 5363 total occurrences across 188 traits
  label_1: 14675 total occurrences across 230 traits
  label_2: 24936 total occurrences across 249 traits
  label_3: 282964 total occurrences across 275 traits

Exported label statistics to results/roles_240/label_stats.csv
